In [90]:
import pymysql as msql
import pandas as pd
import numpy as np

db = msql.connect("localhost","root","david","products")
#sql = 'SELECT product_name, customer_id, sum(product_num) amt FROM products.orders where product_source =1 group by customer_id, product_name'
sql = "SELECT product_name, customer_id, sum(product_num) FROM products.orders where product_source =1 group by product_name, customer_id"
df = pd.read_sql(sql,con=db)



df.to_excel('D:/Project/MyCustomer2.0/forPivot.xlsx')

#数据透视表处理
dt = df.pivot('customer_id','product_name','amt')

# 按照每个产品，获取所有有这个产品的订单，再看其他产品的个数
column_name = list(dt.columns)
output = pd.DataFrame(dt.columns)
for name in column_name:
    non_zero =  dt[dt[name].notnull()]
    non_zero = non_zero.fillna(0)
    merge_sum = non_zero.sum(axis=0)
    output[name]=merge_sum.data

#输出的结果，需要行列转换一下
output.to_excel('D:/Project/MyCustomer2.0/Prod_rel.xlsx')


# 取百分比
data=np.array(output)
data = np.delete(data, 0,axis=1)
proc_data = np.array(data)

for i in range(0,len(data)):
    for j in range(0,len(data)):
               proc_data[i][j] = data[i][j]/data[j][j]
            
# print(proc_data)
df = pd.DataFrame(proc_data.T)
df.index = column_name
df.to_excel('D:/Project/MyCustomer2.0/Prod_rel_converted.xlsx')